# Pool

The first string object put in "yam" is USED by the  
with statement. But because **it is released back into the pool  
afterwards it is reused by the explicit call to sample_queue.get()**.  

Same thing happens with "sam", when **the ObjectPool created insided the  
function is deleted (by the GC) and the object is returned**.

Reference: https://www.itread01.com/content/1537205909.html

In [4]:
import queue

class Objectqueue(object):
    def __init__(self, queue, auto_get=False):
        self._queue = queue
        self.item = self._queue.get() if auto_get else None

    def __enter__(self):
        if self.item is None:
            self.item = self._queue.get()
        return self.item
    
    def __exit__(self, Type, value, traceback):
        if self.item is not None:
            self._queue.put(self.item)
            self.item = None
            
    def __del__(self):
        if self.item is not None:
            self._queue.put(self.item)
            self.item = None

q = queue.Queue()

q.put('yam')
with Objectqueue(q) as obj:
    print(f"Inside with: {obj}")
print(f"Outside with: {q.get()}")

def test_object(q):
    pool = Objectqueue(q, True)
    print(f"Inside function: {pool.item}")

q.put('sam')
test_object(q)
print(f"Outside function: {q.get()}")

if not q.empty():
    print(q.get())

Inside with: yam
Outside with: yam
Inside function: sam
Outside function: sam
